In [2]:
# 
import pandas as pd

df = pd.read_csv('./csvData/NBA_Shot_dist - Sheet2.csv')

# Change column names

In [ ]:
df.columns = ['name', 'team', 'szn', 'type', 'games', '0to8_makePrecent', '8to16_makePrecent', '16to24_makePrecent', '24plus_makePrecent', 'bcMake', '0to8_att', '8to16_att', '16to24_att', '24plus_att', 'bc_att', 'avgDist', 'avgMakeDist', 'avgMissDist']

# Delete irrelevant columns

In [ ]:
del df['type']
del df['szn']
del df['bcMake']

In [2]:
df = df.sort_values('name')

NameError: name 'df' is not defined

# Bring in Salary Data

*  Delete all columns except name and salary
*  Rename two columns 'name' and 'salary'

In [5]:
salary_df = pd.read_csv('playerSalaries.csv')

In [6]:
del salary_df['Unnamed: 0']
del salary_df['2018/19']
del salary_df['2019/20']
del salary_df['2020/21']
del salary_df['2021/22']

salary_df.columns = ['name', 'salary']

# Sort Salary Data

In [7]:
salary_df = salary_df.sort_values('name')

# Merge Data sets 
##### How? ...... inner


In [8]:
df = pd.merge(df, salary_df, how='inner')

# Get X,y 

* get rid of specail characters '$', and ','

In [9]:
import numpy as np

for col in df.select_dtypes([np.object]):
    df[col] = df[col].str.lstrip('$')
    


df = df.replace(regex={'$':'', ',':''})

df = df.loc[df['games'] >= 26]

# add more data
* def_stats
    * defensive win share
* gen_stats
    * age
    * minutes played avg
    * points per game
    * 

In [10]:
def_ws = pd.read_csv('NBA_Shot_dist - def_stats.csv')

def_ws = def_ws.iloc[:, [0,19]]

def_ws.columns = ['name', 'def_ws']

#------------------

def_gen = pd.read_csv('NBA_Shot_dist - gen_stats.csv')
def_gen = def_gen.iloc[:, [0,2,6,7,14,15,16,19,20,21,22,23,24,28]]
def_gen.columns = ['name','age','min','pts','ftm','fta','ftp','reb','ast','tov','stl','blk','pf','pm']

# Merge with main DF

In [11]:
df = pd.merge(df, def_ws, how='inner')
df = pd.merge(df, def_gen, how='inner')

# Delete certain columns

In [12]:
del df['0to8_att']
del df['8to16_att']
del df['16to24_att']
del df['24plus_att']
del df['avgMissDist']
del df['avgDist']
del df['bc_att']

# Get X, y as dataframes
* make salary your 'y'
* remove salary out of the middle of the df
* make 'x' data frame

In [13]:
y = df.iloc[:, 8]
del df['salary']
X = df.iloc[:, 2:23]

# Turn Y into an intger

In [14]:
y = y.astype('int')


# Normalization && Tuning 

* squre (^2) avgMakeDist col

# Train/Test split 70/30

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

# First Run w/ Random Forest Classifer

In [17]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics

""" RANDOM FOREST PREDICTION """
# X, y = make_classification(n_samples=100, n_features=5, n_informative=5, n_redundant=0, random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=120, max_depth=25, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred), " -- Accuracy with Random Forests\n")
print("-------------- Importances per column with RF")
print((clf.feature_importances_), "\n")



0.0673076923076923  -- Accuracy with Random Forests

-------------- Importances per column with RF
[0.05044319 0.05491385 0.05317709 0.05122454 0.04890959 0.05335486
 0.05022871 0.05690085 0.05282702 0.05029502 0.04224432 0.04342287
 0.05187173 0.05433081 0.04956383 0.0459346  0.0431036  0.04327643
 0.04836301 0.05561408] 



# Predictions

In [18]:
#print(clf.predict([[]]))